In [1]:
import pandas as pd
import numpy as np
from environment import Environment
from pandas.tseries.offsets import MonthEnd
import instrument
from portfolio import Portfolio

%load_ext autoreload
%autoreload 2

In [2]:
from historical import HistoricalData
from pandas._libs.tslibs.timestamps import Timestamp, Timedelta

path = "data\\"
env_df = pd.read_csv(path + "Consolidated.csv", header = [0,1], index_col=[0])
targets_df = pd.read_csv(path + "Targets.csv", index_col=[0])
features_df = pd.read_csv(path + "Features.csv", header = [0,1], index_col=[0])


data = HistoricalData(env_df, features_df, targets_df)

In [3]:
date = Timestamp('2006-06-01') + MonthEnd(0)
opt_mat = date + MonthEnd(2)
env_now = Environment(*data.get_env_args(date))
pos_dict = {}
etf_dict = {}

icash = instrument.Cash('USD')
ispy = instrument.Equity('SPY US Equity', 'USD')
my_pf = Portfolio({icash: 200000, ispy: 0})

C = 1 + 0.2/12
K1 = C - 0.02
K2 = C
K3 = C + 0.02
nopt = 100000
ttm=2

specs = [{'name':'SPY Call 1', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*K1},
         {'name':'SPY Call 2', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*K2},
         {'name':'SPY Call 3', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*K3}
         ]

my_pf.buy_options(env_now, specs, [nopt, -nopt*2, nopt], ttm=ttm)



4029.8500000000004

In [4]:
my_pf.pf_units

{<instrument.Cash at 0x25b5ca7b748>: 162946.4898740074,
 <instrument.Equity at 0x25b5ca7bd30>: 0,
 <instrument.Option at 0x25b5ca7bda0>: 100000,
 <instrument.Option at 0x25b5ca7be48>: -200000,
 <instrument.Option at 0x25b5ca7be10>: 100000}

In [5]:
my_pf.get_options()[0].value(env_now)

3.304097194269985

In [6]:
my_pf.calc_value(env_now)

195970.14999999997

In [7]:
pf_vals = []
ul_price = []

for i in range(1, 60):
    # set date to nex month
    date = date + MonthEnd(1)
    opt_mat = date + MonthEnd(2)
    env_now = Environment(*data.get_env_args(date))
    
    # sell current options
    fee = my_pf.sell_options(env_now)
    
    specs = [{'name':'SPY Call 1', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*K1},
         {'name':'SPY Call 2', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*K2},
         {'name':'SPY Call 3', 'ccy':'USD', 'is_call':1, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*K3}
         ]

    fee = my_pf.buy_options(env_now, specs, [nopt, -nopt*2, nopt], ttm=ttm)
    
    # record portf val
    pf_vals.append(my_pf.calc_value(env_now))
    ul_price.append(my_pf.get_asset('SPY US Equity').value(env_now))

4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000000004
4029.8500000

In [8]:
pd.DataFrame(np.array([pf_vals, ul_price]).T)

,0,1
0,202213.700538,133.7196
1,242440.916450,136.6377
2,282075.322462,140.3260
3,314994.267133,144.7487
4,354339.217494,147.6270
5,390516.586198,149.6006
6,434639.050780,151.8506
7,407012.299323,148.8717
8,425360.381075,150.5989
9,451800.604133,157.2698


In [9]:
my_pf.get_options()[1].T

Timestamp('2011-07-31 00:00:00')

In [10]:
env_now.curves['USD'].values

array([0.0009, 0.0009])

In [11]:
pf_vals = np.array(pf_vals)
pf_rets = pf_vals[1:]/pf_vals[0:-1] - 1

m = pf_rets.mean()*12
s = pf_rets.std()*np.sqrt(12)
print(m,s)
print(m/s)

0.2653758180369217 0.22299098068900552
1.1900742228091645


In [12]:
my_pf.pf_units

{<instrument.Cash at 0x25b5ca7b748>: 613168.902911566,
 <instrument.Equity at 0x25b5ca7bd30>: 0,
 <instrument.Option at 0x25b5caa7630>: 100000,
 <instrument.Option at 0x25b5cab9048>: -200000,
 <instrument.Option at 0x25b5caaaeb8>: 100000}